# Example for automated image genration using orthophtos and shape-files


## 1.) Input files:
- specify file paths
- choose webdriver (tested with chrome,firefox,safrie). (Default=chrome, download via: https://chromedriver.chromium.org/downloads, copy file to: copy to usr/local/bin/


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


script_dir = '/Users/flo/Proj/sat_hedge_detection/Examples/'
shape_file_path = '../data/Biotopkartierung/Biotopkartierung.shp'
wmts = "http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg"

## choose driver
args_driver = 'chrome'
args_driver_path = '/usr/local/bin/chromedriver'


print("Done")

Done


## 2.) Load packages and functions



In [143]:
import os
import io
from PIL import Image

import geopandas as gpd

import folium
from folium.folium import Map
from folium.features import DivIcon

from pyproj import Transformer

print('Start...')

def getText(temp_biotop):
    temp_nummer = temp_biotop['Nummer'][temp_biotop['Nummer'].keys()[0]]
    temp_gemnr = temp_biotop['gemnr'][temp_biotop['gemnr'].keys()[0]]
    temp_biotoptyp = temp_biotop['biotoptyp'][temp_biotop['biotoptyp'].keys()[0]]
    temp_bezeich = temp_biotop['bezeich'][temp_biotop['bezeich'].keys()[0]]
    text = "Nummer: " + str(temp_nummer) + "\n" + "GemNr: " + str(temp_gemnr) + "\n" + "Biotoptyp: " + str(
        temp_biotoptyp) + "\n" + "Bezeichnung: " + str(temp_bezeich)
    return text

def style_fcn(x):
    return {'stroke':True,'color': '#FF0000', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}

def style_fnc_mask(x):
    return {'stroke':True,'color': '#FFFFFF', 'fillColor': '#FFFFFF','opacity':1, 'weight':10, 'line_cap':'round', 'fill':True, 'fillOpacity': 1}

def style_fcn_hull(x):
    return {'stroke':True,'color': '#008FFF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}


def style_fcn_env(x):
    return {'stroke':True,'color': '#F700FF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}

def create_info_textfile(biotop):

    temp_nummer = temp_biotop['Nummer'][temp_biotop['Nummer'].keys()[0]]
    temp_gemnr = temp_biotop['gemnr'][temp_biotop['gemnr'].keys()[0]]
    temp_biotoptyp = temp_biotop['biotoptyp'][temp_biotop['biotoptyp'].keys()[0]]
    temp_bezeich = temp_biotop['bezeich'][temp_biotop['bezeich'].keys()[0]]
    temp_bio_katgem = temp_biotop['bio_katgem'][temp_biotop['bio_katgem'].keys()[0]]
    temp_bio_subnr = temp_biotop['bio_subnr'][temp_biotop['bio_subnr'].keys()[0]]
    temp_bezeichnun = temp_biotop['bezeichnun'][temp_biotop['bezeichnun'].keys()[0]]
    temp_rechtl_Sch = temp_biotop['rechtl_Sch'][temp_biotop['rechtl_Sch'].keys()[0]]
    link1 = temp_biotop['Link'][temp_biotop['Link'].keys()[0]]
    link2 = temp_biotop['Link_ext'][temp_biotop['Link_ext'].keys()[0]]
    #link1: http://anwendung/ins/biotop/display.do?id=1175590
    #link2: https://service.salzburg.gv.at/ins/biotop/disp...
    # =>    https://service.salzburg.gv.at/ins/biotop/ + display.do?id=1175590
    # link immer 7-Stellig? Sonst nach = suchen
    ID = link1[-7:]
    link_public = "https://service.salzburg.gv.at/ins/biotop/display.do?id="+str(ID)
    link_intern = "https://portal.salzburg.gv.at/ins/biotop/display.do?id="+str(ID)

    temp_path = args_output_dir + 'bio_' + temp_nummer + '/' + temp_nummer + "_info.txt"

    output_file = open(temp_path, 'w')
    output_file.write("Biotop Nummer: "+ str(temp_nummer) + "\n")
    output_file.write("Biotop Kat Gem: "+ str(temp_bio_katgem) + "\n")
    output_file.write("Biotop Sub Nr: "+ str(temp_bio_subnr) + "\n")
    output_file.write("Gemeinde Nummer: "+ str(temp_gemnr)+ "\n")
    output_file.write("Biotop Typ: "+ str(temp_biotoptyp) + "\n")
    output_file.write("Biotop Bezeichnung: "+ str(temp_bezeich) + "\n")
    output_file.write("Biotop Bezeichnung(2): "+ str(temp_bezeichnun) + "\n")
    output_file.write("Biotop §: "+ str(temp_rechtl_Sch) + "\n")
    output_file.write("Biotop Link-Public: "+ link_public + "\n")
    output_file.write("Biotop Link-Public: "+ link_intern + "\n")
    output_file.write("Biotop Location: "+ str(biotop_center(biotop)) + "\n")
    output_file.write("Biotop Surface (approx m2): {:02.2f}\n".format(biotop_surface(biotop)))
    output_file.close()
    
def biotop_surface(biotop):
    #check?
    temp = temp_biotop['geometry'].to_crs("EPSG:3395")\
               .map(lambda p: p.area)
    return temp[temp.keys()[0]]*0.450


def biotop_center(temp_biotop):
    #get biotop pos
    center_temp = temp_biotop.geometry.centroid
    key = center_temp.keys()[0]
    return transformer.transform(center_temp[key].x,center_temp[key].y)

def biotop_current_map(temp_location, text, BIOTOP_BORDER, BIOTOP_MASK ,BIOTOP_DESCRIPTION, case):
    BIOTOP_SCALE = BIOTOP_CONVEX = BIOTOP_ENV = 0

    m_temp = Map(tiles=None, location=[temp_location[1], temp_location[0]],
                 zoom_start=20,
                 prefer_canvas=True,
                 no_touch=True,
                 disable_3d=True,
                 attr="test",
                 control_scale=True,
                 zoom_control=False)

    atlas = folium.raster_layers.WmsTileLayer(url=wmts, layers="bmaporthofoto30cm",
                                              fmt='image/png',
                                              transparent=True,
                                              overlay=True).add_to(m_temp)

    if BIOTOP_BORDER:
        case = 'B_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_scaled,style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_convex_hull, style_function=style_fcn_hull).add_to(m_temp)
    # folium.GeoJson(temp_biotop_envelope, style_function=style_fcn_env).add_to(m_temp)

    if BIOTOP_MASK:
        case = 'M_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fnc_mask).add_to(m_temp)

    ## WRITE BIOTOP DESCIBTION IN IMAGE
    if BIOTOP_DESCRIPTION:
        folium.map.Marker([temp_location[1] + 0.001, temp_location[0] - 0.001],
                    icon=DivIcon(
                        icon_size=(150, 36),
                        icon_anchor=(0, 0),
                        html='<div style="font-size: 10pt;"><span style="color: #ff0000;">%s</span></div>' % text)).add_to(m_temp)
    if BIOTOP_SCALE:
        temp_biotop_scaled = temp_biotop.scale(xfact=2, yfact=2, zfact=0, origin='center')
    if BIOTOP_CONVEX:
        temp_biotop_convex_hull = temp_biotop.convex_hull
    if BIOTOP_ENV:
        temp_biotop_envelope = temp_biotop.envelope
    save_current_biotop2(m_temp, bio_i, case)
    #return m_temp

def save_current_biotop(m_temp, bio_number):
    png = m_temp._to_png()
    out = open(os.path.join(script_dir, '../data/output_biotop/'+bio_number+'.png'), 'wb')
    bytes_written = out.write(png)
    out.close()

def create_bio_path(bio_i):
    temp_path = args_output_dir + 'bio_' + bio_i + '/'
    if not os.path.exists(temp_path):
        os.makedirs(temp_path)
        return True
    return False

def save_current_biotop2(m_temp, bio_number,case):
    img_data = m_temp._to_png()
    img = Image.open(io.BytesIO(img_data))
    temp_path = args_output_dir + 'bio_' + bio_number + '/'
    img.save(os.path.join(script_dir, temp_path + case + '_' + bio_number + '.png'))
    
    
print('Load: Biotopkartierung ', shape_file_path)
nReserve = gpd.read_file(os.path.join(script_dir, shape_file_path))

print('Load: wmts layer:' ,wmts)

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')


print("Done")

Start...
Load: Biotopkartierung  ../data/Biotopkartierung/Biotopkartierung.shp
Load: wmts layer: http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg
Start: webdriver  chrome
Done


# 3.) Main script
- generates images (png) based on orthophtos and biotop shapefile 
     currently: biotoptyp=='8.1.1.1 + biotoptyp=='8.1.1.2
- create path: sat_hedge_detection/data/output_biotop_dir/

### Parameter to specify:
-  currently: todo

## 3.1) Extract all Biotope regions

In [151]:
#script_dir = os.path.dirname(__file__)
#script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'

args_shapefile = '../data/Biotopkartierung/Biotopkartierung.shp'
#args_biotypes = ['8.1.1.1', '8.1.1.2']
args_biotypes = typ_list


args_wmts = 'http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg'
args_text = True
args_output_dir = '../data/output_biotop_dir_test/'


biotop_dict = {}
for i in range(0,len(args_biotypes)):
    str_nummer = "\'" + args_biotypes[i] + "\'"
    query_str = "biotoptyp==" + str(str_nummer)
    biotop_dict[i] = nReserve.query(query_str)

print('Load: bmaporthofoto30cm')
wmts = args_wmts
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')


transformer = Transformer.from_crs(31258, 4326, always_xy=True)  # 1frpm 2to


len_totoal = old_i =  0
for iter_bio in range(0,len(args_biotypes)):
    len_totoal = len_totoal + biotop_dict[iter_bio].shape[0]

#for biotop_dict[iter_bio] in biotop_dict:
for iter_bio in range(0,len(args_biotypes)):
    biotop_key_nummer = biotop_dict[iter_bio]['Nummer']
    biotop_keys = biotop_key_nummer.keys()
    for i in range(0, biotop_dict[iter_bio].shape[0]-1):
        key_i = biotop_keys[i]
        bio_i = biotop_dict[iter_bio]['Nummer'][key_i]
        str_nummer = "\'" + bio_i + "\'"
        query_str = "Nummer==" + str(str_nummer)
        temp_biotop = biotop_dict[iter_bio].query(query_str)
        temp_location = biotop_center(temp_biotop)
        
        
        
        PATH_NEW = create_bio_path(bio_i)
        if PATH_NEW:
            print("Process Biotop: ", i+old_i, "/", len_totoal, " ", bio_i, " type: ", args_biotypes[iter_bio])
            #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False, case=1)
            
            ## Raw Bio Image
            #biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
            #            case=args_biotypes[iter_bio][-1])
            
            ## Red Border File
            biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                        case=args_biotypes[iter_bio][-1])
            
            ## Bio Mask (for Training/Labeling)
            #biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
            #            case=args_biotypes[iter_bio][-1])
            
            if args_text: create_info_textfile(temp_biotop)
        else:
            print("Already Existing Biotop: ", i+old_i, "/", len_totoal, " ", bio_i)
        break
        
    old_i = old_i + i
print("Done.")
driver.quit()

Load: bmaporthofoto30cm
Start: webdriver  chrome
Already Existing Biotop:  0 / 74655   555090084
Already Existing Biotop:  0 / 74655   564050088
Already Existing Biotop:  0 / 74655   555120087
Already Existing Biotop:  0 / 74655   555120190
Already Existing Biotop:  0 / 74655   564150254
Already Existing Biotop:  0 / 74655   564110042
Already Existing Biotop:  0 / 74655   571160209
Already Existing Biotop:  0 / 74655   564050047
Already Existing Biotop:  0 / 74655   564050029
Already Existing Biotop:  0 / 74655   564050106
Already Existing Biotop:  0 / 74655   564050004
Already Existing Biotop:  0 / 74655   565070116
Already Existing Biotop:  0 / 74655   572100348
Already Existing Biotop:  0 / 74655   555120137
Already Existing Biotop:  0 / 74655   564100001
Already Existing Biotop:  0 / 74655   573061820
Already Existing Biotop:  0 / 74655   563170051
Already Existing Biotop:  0 / 74655   564110215
Already Existing Biotop:  0 / 74655   564130137
Already Existing Biotop:  0 / 74655   5

Process Biotop:  0 / 74655   571250037  type:  9.1.1
Process Biotop:  0 / 74655   553150075  type:  9.1.2
Process Biotop:  0 / 74655   555120044  type:  9.1.3
Process Biotop:  0 / 74655   572101549  type:  9.10
Process Biotop:  0 / 74655   572100074  type:  9.10.2
Process Biotop:  0 / 74655   571090080  type:  9.10.3
Process Biotop:  0 / 74655   553200044  type:  9.11.1.1
Process Biotop:  0 / 74655   571160197  type:  9.12.1.1
Process Biotop:  0 / 74655   572100028  type:  9.14.1
Process Biotop:  0 / 74655   566010089  type:  9.2.1
Process Biotop:  0 / 74655   571160188  type:  9.2.1.1.1
Process Biotop:  0 / 74655   571140084  type:  9.2.1.1.2
Process Biotop:  0 / 74655   565140005  type:  9.2.2.1
Process Biotop:  0 / 74655   571090039  type:  9.2.2.2
Process Biotop:  0 / 74655   571140001  type:  9.2.2.2.1
Process Biotop:  0 / 74655   571140013  type:  9.2.2.2.2
Process Biotop:  0 / 74655   572100363  type:  9.2.2.2.3
Process Biotop:  0 / 74655   572160820  type:  9.2.2.3
Process Biot

## 3.2) Extract single biotope

In [149]:
#script_dir = os.path.dirname(__file__)
#script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'

args_shapefile = '../data/Biotopkartierung/Biotopkartierung.shp'
args_biotypes = ['8.1.1.1', '8.1.1.2']
args_wmts = 'http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg'
args_text = True
args_output_dir = '../data/output_biotop_dir/'

##
target_biotope = '565030179'
##



print('Load: bmaporthofoto30cm')
wmts = args_wmts
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')

transformer = Transformer.from_crs(31258, 4326, always_xy=True)  # 1frpm 2to

str_nummer = "\'" + target_biotope + "\'"
query_str = "Nummer==" + str(str_nummer)
temp_biotop = nReserve.query(query_str)
bio_i = target_biotope
temp_location = biotop_center(temp_biotop)
#print(temp_biotop)

temp_typ = temp_biotop["biotoptyp"]
temp_key = (temp_typ.keys()[0])

PATH_NEW = create_bio_path(target_biotope)
if PATH_NEW:
    print("Process Biotop: ", target_biotope, " type: ", temp_typ[temp_key])
    #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False, case=1)

    ## Raw Bio Image
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
                case=temp_typ[temp_key][-1])

    ## Red Border File
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                case=temp_typ[temp_key][-1])

    ## Bio Mask (for Training/Labeling)
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
                case=temp_typ[temp_key][-1])

    if args_text: create_info_textfile(temp_biotop)
else:
    print("Already Existing Biotop: ", target_biotope)

print("Done.")
driver.quit()

Load: bmaporthofoto30cm
Start: webdriver  chrome
Already Existing Biotop:  565030179
Done.


### Get all biotio nr corresbonding to §24

In [17]:
nReserve["rechtl_Sch"]
target = "§24"
str_nummer = "\'" + target + "\'"
query_str = "rechtl_Sch==" + str(str_nummer)
biotope = nReserve.query(query_str)

In [34]:
print("Number of biotops beloning to ", target, ":", len(biotope), "/", len(nReserve))

Number of biotops beloning to  §24 : 41069 / 82802


In [46]:
biotop_keys = biotope["Nummer"].keys()

In [133]:
typ_list = []
bez_list = []
typ_bez = {}
count = 0
for temp_key in biotop_keys:
    temp_typ = biotope["biotoptyp"][temp_key]
    temp_bez = biotope["bezeichnun"][temp_key]
    if temp_typ not in typ_list:
        typ_list.append(temp_typ)
        bez_list.append(temp_bez)
        typ_bez[count] = {'typ': temp_typ, 'bez': temp_bez}
        count = count+1

In [148]:
bez_list
#0,2

['BT Staudenreicher Hochgebirgsrasen',
 'Gebirgsbäche',
 'BT Karbonat-Latschen-Buschwald',
 'Karbonatschutthalden der Hochlagen',
 'Kleinseggenrieder',
 'Naturnahe Teiche und Weiher',
 'Feucht- und Nassgrünland nährstoffarmer Standorte',
 'SUBTYP Grauerlenau, fragmentarisch ausgebildet',
 'Block- und Schutthalden',
 'BT Firn- und Altschneefeld',
 'BT Naturnaher Tümpel',
 'Hochmoore',
 'BT Großröhricht an Stillgewässer und Landröhricht',
 'Hochgebirgsbäche',
 'Ufergehölzstreifen',
 'BT Latschenmoorwald',
 'Karbonatschneetälchen und -schneeböden',
 'BT Horstiges Großseggenried',
 'BT Alpine bis nivale Polsterfluren und Rasenfragmente über Karbonat',
 'BT Geschlossener Hochgebirgs-Karbonatrasen',
 'Quellen',
 'Karbonatfelswände mit Felsspaltenvegetation',
 'BT Rasiges Großseggenried',
 'Hügellandbäche',
 'Hochstaudenfluren der tieferen Lagen',
 'BT Karbonat-Rotföhrenwald',
 'Grünland feuchter bis nasser Standorte',
 'Halbtrockenrasen',
 'BT Frische basenarme Magerweide der tiefen Lagen',


In [56]:
biotope

,Nummer,bio_katgem,bio_subnr,bio_versio,schluessel,gemnr,bezeich,rechtl_Sch,biotoptypg,biotoptyp,bezeichnun,Link,Link_ext,ueberarbei,SHAPE_STAr,SHAPE_STLe,geometry
0,555090072,55509.0,0072,1,5.550901e+09,50424,Rostseggenrasen nördlich Schwarzkogel,§24,4,4.1.4,BT Staudenreicher Hochgebirgsrasen,http://anwendung/ins/biotop/display.do?id=1099790,https://service.salzburg.gv.at/ins/biotop/disp...,None,16007.907913,1331.355955,"POLYGON ((436068.750 258655.984, 436071.188 25..."
1,555120190,55512.0,0190,1,5.551202e+09,50425,Pichler Waldbach,§24,1,1.3.2.2,Gebirgsbäche,http://anwendung/ins/biotop/display.do?id=1102470,https://service.salzburg.gv.at/ins/biotop/disp...,None,567.168533,393.914902,"POLYGON ((442949.344 258722.906, 442953.531 25..."
2,555090033,55509.0,0033,1,5.550900e+09,50424,Bach beim Fischteich südlich Brand,§24,1,1.3.2.2,Gebirgsbäche,http://anwendung/ins/biotop/display.do?id=1099751,https://service.salzburg.gv.at/ins/biotop/disp...,None,229.486168,245.958244,"POLYGON ((437600.375 258721.781, 437609.719 25..."
3,571250037,57125.0,0037,1,5.712500e+09,50609,Kalk-Latschenbestand NW Riedlalm,§24,9,9.1.1,BT Karbonat-Latschen-Buschwald,http://anwendung/ins/biotop/display.do?id=1148772,https://service.salzburg.gv.at/ins/biotop/disp...,None,946249.611183,11488.886380,"POLYGON ((402583.260 257907.267, 402597.823 25..."
4,571160022,57116.0,0022,1,5.711600e+09,50619,Schuttfläche in SO vom Ahlhorn,§24,10,10.5.2.1,Karbonatschutthalden der Hochlagen,http://anwendung/ins/biotop/display.do?id=1143121,https://service.salzburg.gv.at/ins/biotop/disp...,None,14636.258896,879.450673,"POLYGON ((417647.375 258391.672, 417638.562 25..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82792,565290627,56529.0,0627,1,5.652906e+09,50323,Erlenbruchwaldrest W Aug,§24,9,9.3.1,BT Erlenbruch- und -sumpfwald,http://anwendung/ins/biotop/display.do?id=1207531,https://service.salzburg.gv.at/ins/biotop/disp...,None,1744.424194,182.157999,"POLYGON ((433185.127 316497.538, 433172.424 31..."
82797,565040249,56504.0,0249,1,5.650402e+09,50304,Teichkette SW Berndorf,§24,1,1.4.3.2.2,BT Oligotropher naturnaher Teich und Weiher ti...,http://anwendung/ins/biotop/display.do?id=1232145,https://service.salzburg.gv.at/ins/biotop/disp...,None,330.432358,90.935539,"POLYGON ((429266.376 316823.699, 429267.799 31..."
82798,564070088,56407.0,0088,1,5.640701e+09,50329,Teich N Wimpassing,§24,1,1.4.3.4.1,BT Poly- bis hypertropher Teich und Weiher,http://anwendung/ins/biotop/display.do?id=1232146,https://service.salzburg.gv.at/ins/biotop/disp...,None,399.346596,88.079189,"POLYGON ((418895.734 313461.107, 418908.963 31..."
82799,565160151,56516.0,0151,1,5.651602e+09,50304,Großseggensaum um Teiche W Grub,§24,2,2.2.1.2.1,"SUBTYP Rasiges Großseggenried, typischer Subtyp",http://anwendung/ins/biotop/display.do?id=1232147,https://service.salzburg.gv.at/ins/biotop/disp...,None,858.400269,246.038794,"POLYGON ((428297.909 315909.749, 428300.016 31..."
